In [1]:
from scipy.io import wavfile
import os
import numpy as np
import pandas as pd

SETUP

In [2]:
# Variables you may change
numberSpikes = 20 # number of integer multiples of the fundamental frequency to record (including 0 Hz).
FFTSize = 8192 # have to use a multiple of 2 for optimal speed.

#open directory with the samples in it.
os.chdir('./Training_data/')

# next, make a NP array of every wav file in the directory
SampleList = np.array(os.listdir())
SampleList = SampleList[ np.char.endswith(SampleList, '.wav') ] #ignore everything that's not a .wav file.
print(SampleList)

print('SampleList size ',np.shape(SampleList))
print('Index size ', np.shape([ SampleList , ['Pwr','dB'] ]))
print(int(len(SampleList)))

#make a fresh blank series.
SampleSpectra = pd.DataFrame(columns = np.arange(numberSpikes))

['01 - [Prologue]_1.wav']
SampleList size  (1,)
Index size  (2,)
1


CORE LOOP

In [3]:
for SampleName in SampleList:
    SampleRate, Audiodata = wavfile.read(SampleName)
    # spectrum
    from scipy.fft import fft # fourier transform
    m = len(Audiodata) 
    #AudioFreq = fft(Audiodata,n=FFTSize,axis=0)
    AudioFreq = fft(Audiodata[:,1]*np.hanning(np.shape(Audiodata)[0]),n=FFTSize,axis=0) #with a Hanning window
    AudioFreq = AudioFreq[0:int(np.ceil((np.shape(AudioFreq)[0])/2.0))] #Left half of the spectrum
    MagFreq = np.abs(AudioFreq) # Magnitude
    MagFreq = MagFreq / float(m)
    # power spectrum
    MagFreq = MagFreq**2
    if m % 2 > 0: # ffte odd 
        MagFreq[1:len(MagFreq)] = MagFreq[1:len(MagFreq)] * 2
    else:# fft even
        MagFreq[1:len(MagFreq) -1] = MagFreq[1:len(MagFreq) - 1] * 2 
    # HPS (Harmonic Product Spectrum) Algorithm to determine fundamental frequency
    HPCSize = int(np.ceil(len(MagFreq))/5.0 + 1)
    MagFreq2 = MagFreq.copy().reshape(int(np.ceil(len(MagFreq))/2.0),2).mean(1)[0:HPCSize]
    MagFreq3 = np.concatenate([MagFreq.copy(),np.zeros(3 - int(len(MagFreq)) % 3)]).reshape(int(np.ceil(len(MagFreq))/3.0 + 1),3).mean(1)[0:HPCSize]
    HPC = MagFreq[0:HPCSize]*MagFreq2*MagFreq3
    FFLocation = np.argmax(HPC)
    # Have to ensure the fundamental frequency is accurate. Take loudest spike, get its approximate ratio, round to nearest int
    MaxLoc = np.argmax(MagFreq)
    freqAxis = np.arange(0,int(np.ceil((m+1)/2.0)), int(np.ceil((m+1)/2.0))*2/FFTSize) * (SampleRate / m);
    MaxRatio = np.round(freqAxis[MaxLoc]/freqAxis[FFLocation])
    FFreq = freqAxis[MaxLoc]/MaxRatio
    MagFreqLog = 10*np.log10(MagFreq)
    FFreqLoc = np.argmin(np.abs(freqAxis - FFreq))
    # If the frequency on either side of the "spike" is actually louder, make that the new spike.
    FFreqLoc = FFreqLoc - ((MagFreqLog <= np.roll(MagFreqLog,1)) & (np.roll(MagFreqLog,-1) <= np.roll(MagFreqLog,1))) #if the left entry is >= the entry and the right entry, then make the left entry the new spike location
    FFreqLoc = FFreqLoc + ((MagFreqLog <= np.roll(MagFreqLog,-1)) & (np.roll(MagFreqLog,1) <= np.roll(MagFreqLog,-1)))  #if the right entry is >= the entry and the left entry, then make the right entry the new spike location
    Multiples = [FFreq*i for i in np.arange(0,numberSpikes)]
    MultiplesLoc = [np.argmin(np.abs(freqAxis - i)) for i in Multiples]
    # Next, make an integer ratio-indexed list of the amplitudes!
    MultiplesMag = MagFreq[MultiplesLoc]
    MultiplesdB = MagFreqLog[MultiplesLoc]
    # Write these magnitudes/dBs to the Pandas array! (Index = SampleName)
    print(np.vstack([MultiplesMag, MultiplesdB]))
    print([[SampleName],['Pwr','dB']])
    tempIndex = pd.MultiIndex.from_product([[SampleName],['Pwr','dB']])
    SampleRow = pd.DataFrame(np.vstack([MultiplesMag, MultiplesdB]), index = tempIndex, columns = np.arange(0,numberSpikes))
    SampleSpectra = pd.concat([SampleSpectra, SampleRow])

[[ 8.24749141e-02  3.46080249e-01  1.24053034e+03  5.05324902e-01
   3.55408349e-01  7.43261023e-02  8.41128443e-02  2.99093037e-02
   2.22068712e-02  1.71220618e-02  1.35146583e-02  1.05372968e-02
   7.97451996e-03  7.84191772e-03  5.42737697e-03  5.65548568e-03
   4.38594088e-03  4.23322675e-03  3.19160452e-03  3.46324621e-03]
 [-1.08367813e+01 -4.60823185e+00  3.09360739e+01 -2.96429300e+00
  -4.49272375e+00 -1.12885864e+01 -1.07513768e+01 -1.52419370e+01
  -1.65351263e+01 -1.76644394e+01 -1.86919493e+01 -1.97727079e+01
  -2.09829545e+01 -2.10557772e+01 -2.26541001e+01 -2.24753009e+01
  -2.35793723e+01 -2.37332847e+01 -2.49599093e+01 -2.46051663e+01]]
[['01 - [Prologue]_1.wav'], ['Pwr', 'dB']]


CSV FILE OUTPUT

In [4]:
SampleSpectra.to_csv('SampleSpectra.csv')
#with io.open('SampleSpectra.csv','w') as outputFile:
#    outputFile.write(SampleSpectra)

print('The first few samples\' spectra for your viewing pleasure:')
print(SampleSpectra.head())

The first few samples' spectra for your viewing pleasure:
                                     0         1            2         3   \
(01 - [Prologue]_1.wav, Pwr)   0.082475  0.346080  1240.530344  0.505325   
(01 - [Prologue]_1.wav, dB)  -10.836781 -4.608232    30.936074 -2.964293   

                                    4          5          6          7   \
(01 - [Prologue]_1.wav, Pwr)  0.355408   0.074326   0.084113   0.029909   
(01 - [Prologue]_1.wav, dB)  -4.492724 -11.288586 -10.751377 -15.241937   

                                     8          9          10         11  \
(01 - [Prologue]_1.wav, Pwr)   0.022207   0.017122   0.013515   0.010537   
(01 - [Prologue]_1.wav, dB)  -16.535126 -17.664439 -18.691949 -19.772708   

                                     12         13         14         15  \
(01 - [Prologue]_1.wav, Pwr)   0.007975   0.007842   0.005427   0.005655   
(01 - [Prologue]_1.wav, dB)  -20.982955 -21.055777 -22.654100 -22.475301   

                             